In [8]:
import face_recognition
import cv2
import numpy as np
import os
import pickle
import dlib

In [9]:
# Load an image file and create a face encoding
aashish_image = face_recognition.load_image_file(r"C:\Users\aashi\OneDrive\Desktop\JUPTER PHOTO\AAshish\aashish (1).jpg")
aashish_face_encoding = face_recognition.face_encodings(aashish_image)[0]
print("aashish")

bikash_image = face_recognition.load_image_file(r"C:\Users\aashi\OneDrive\Desktop\JUPTER PHOTO\bikash\bikash (5).jpg")
bikash_face_encoding = face_recognition.face_encodings(bikash_image)[0]
print("bikash")

Prof_Dr_Sukanya_Mukherjee_image = face_recognition.load_image_file(r"C:\Users\aashi\OneDrive\Desktop\JUPTER PHOTO\sukanya Ma'am\Prof_Dr_Sukanya_Mukherjee (1).jpg")
Prof_Dr_Sukanya_Mukherjee_face_encoding = face_recognition.face_encodings(Prof_Dr_Sukanya_Mukherjee_image)[0]
print("Prof_Dr_Sukanya_Mukherjee")


# Create arrays of known face encodings and their names
known_face_encodings = [
    aashish_face_encoding,
    bikash_face_encoding,
    Prof_Dr_Sukanya_Mukherjee_face_encoding,
]
known_face_names = [
    "aashish",
    "bikash",
    "Prof_Dr_Sukanya_Mukherjee",
]

# Create a dictionary to store encodings and names
data = {
    "encodings": known_face_encodings,
    "names": known_face_names,
}

# Save the data to a pickle file
with open("face_data.pkl", "wb") as file:
    pickle.dump(data, file)

print("Face data saved to 'face_data.pkl'")

aashish
bikash
Prof_Dr_Sukanya_Mukherjee
Face data saved to 'face_data.pkl'


In [10]:
# Load known face encodings
with open("face_data.pkl", "rb") as file:
    face_enco = pickle.load(file)

loaded_face_encodings = face_enco["encodings"]
loaded_face_names = face_enco["names"]

print("Loaded face encodings and names:")
print(loaded_face_names)
#face_enco = open('face_data.pkl', 'rb')
#known_face_encodings, known_face_names = pickle.load(face_enco)
#face_enco.close()

# Initialize dlib's face detector
detector = dlib.get_frontal_face_detector()

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Parameters for Shi-Tomasi corner detection
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

def detect_blink(good_new, good_old, threshold=2.0):
    # Calculate the displacement vectors
    displacement = np.linalg.norm(good_new - good_old, axis=1)
    # Count the number of points with significant displacement
    count = np.sum(displacement > threshold)
    return count > len(displacement) * 0.1  # Adjust threshold as needed

# Initialize video capture
video_capture = cv2.VideoCapture(0)
process_this_frame = True

# Initialize variables for blink detection
old_gray = None
p0 = None

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Resize frame to 1/4 size for faster face recognition processing
    small_frame = frame

    # Convert the image from BGR color (OpenCV) to RGB color (face_recognition)
    im_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Convert the image to grayscale for blink detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if process_this_frame:
        # Find all face locations and encodings in the current frame
        face_locations = face_recognition.face_locations(im_rgb)
        print(face_locations)
        face_encodings = face_recognition.face_encodings(im_rgb, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # Compare faces in the current frame with known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Calculate face distances
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                if face_distances[best_match_index] > 0.5:
                    name = "Unknown"
                else:
                    name = known_face_names[best_match_index]

            face_names.append(name)

        # Detect initial points to track if old_gray is None
        if old_gray is None:
            faces = detector(gray)
            for face in faces:
                x, y, w, h = (face.left(), face.top(), face.width(), face.height())
                roi_gray = gray[y:y + h, x:x + w]
                p0 = cv2.goodFeaturesToTrack(roi_gray, mask=None, **feature_params)
                if p0 is not None:
                    p0[:, 0, 0] += x  # Adjust x coordinates
                    p0[:, 0, 1] += y  # Adjust y coordinates
            old_gray = gray.copy()

    if p0 is not None:
        # Calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
        # yaha niche ke line mai error aarha tha q ke a,b,c .... integer nhi tha. So we convert this to int()
            cv2.line(frame, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
            cv2.circle(frame, (int(a), int(b)), 5, (0, 0, 255), -1)

        # Check for liveness
        if detect_blink(good_new, good_old):
            cv2.putText(frame, "Live", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Fake", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Update the previous frame and previous points
        old_gray = gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

# Release handle to the webcam and close any open windows
video_capture.release()
cv2.destroyAllWindows()


Loaded face encodings and names:
['aashish', 'bikash', 'Prof_Dr_Sukanya_Mukherjee']
